# Pathway enrichment

* Using `msigdbr` package in R to download GSEA pathway gene sets
* initially download Hallmark gene sets

In [1]:
#set wd
getwd()
setwd('/scratch/user/s4436039/scdata/Pathway-sets')
getwd()

[1] "/scratch/user/s4436039/Nikita-project"

[1] "/scratch/user/s4436039/scdata/Pathway-sets"

In [2]:
library(msigdbr)

In [3]:
msigdbr_collections()

gs_cat,gs_subcat,num_genesets
<chr>,<chr>,<int>
C1,,299
C2,CGP,3384
C2,CP,29
C2,CP:BIOCARTA,292
C2,CP:KEGG,186
C2,CP:PID,196
C2,CP:REACTOME,1615
C2,CP:WIKIPATHWAYS,664
C3,MIR:MIRDB,2377


In [4]:
msigdbr_species()

species_name,species_common_name
<chr>,<chr>
Anolis carolinensis,"Carolina anole, green anole"
Bos taurus,"bovine, cattle, cow, dairy cow, domestic cattle, domestic cow, ox, oxen"
Caenorhabditis elegans,NA
Canis lupus familiaris,"dog, dogs"
Danio rerio,"leopard danio, zebra danio, zebra fish, zebrafish"
Drosophila melanogaster,fruit fly
Equus caballus,"domestic horse, equine, horse"
Felis catus,"cat, cats, domestic cat"
Gallus gallus,"bantam, chicken, chickens, Gallus domesticus"


In [5]:
#download human hallmark gene sets
h_gene_sets = msigdbr(species = "human", category = "H")
head(h_gene_sets)

gs_cat,gs_subcat,gs_name,gene_symbol,entrez_gene,ensembl_gene,human_gene_symbol,human_entrez_gene,human_ensembl_gene,gs_id,gs_pmid,gs_geoid,gs_exact_source,gs_url,gs_description
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
H,,HALLMARK_ADIPOGENESIS,ABCA1,19,ENSG00000165029,ABCA1,19,ENSG00000165029,M5905,26771021,,,,Genes up-regulated during adipocyte differentiation (adipogenesis).
H,,HALLMARK_ADIPOGENESIS,ABCB8,11194,ENSG00000197150,ABCB8,11194,ENSG00000197150,M5905,26771021,,,,Genes up-regulated during adipocyte differentiation (adipogenesis).
H,,HALLMARK_ADIPOGENESIS,ACAA2,10449,ENSG00000167315,ACAA2,10449,ENSG00000167315,M5905,26771021,,,,Genes up-regulated during adipocyte differentiation (adipogenesis).
H,,HALLMARK_ADIPOGENESIS,ACADL,33,ENSG00000115361,ACADL,33,ENSG00000115361,M5905,26771021,,,,Genes up-regulated during adipocyte differentiation (adipogenesis).
H,,HALLMARK_ADIPOGENESIS,ACADM,34,ENSG00000117054,ACADM,34,ENSG00000117054,M5905,26771021,,,,Genes up-regulated during adipocyte differentiation (adipogenesis).
H,,HALLMARK_ADIPOGENESIS,ACADS,35,ENSG00000122971,ACADS,35,ENSG00000122971,M5905,26771021,,,,Genes up-regulated during adipocyte differentiation (adipogenesis).


In [7]:
unique(h_gene_sets$gs_name)

[1] "HALLMARK_ADIPOGENESIS"                     
 [2] "HALLMARK_ALLOGRAFT_REJECTION"              
 [3] "HALLMARK_ANDROGEN_RESPONSE"                
 [4] "HALLMARK_ANGIOGENESIS"                     
 [5] "HALLMARK_APICAL_JUNCTION"                  
 [6] "HALLMARK_APICAL_SURFACE"                   
 [7] "HALLMARK_APOPTOSIS"                        
 [8] "HALLMARK_BILE_ACID_METABOLISM"             
 [9] "HALLMARK_CHOLESTEROL_HOMEOSTASIS"          
[10] "HALLMARK_COAGULATION"                      
[11] "HALLMARK_COMPLEMENT"                       
[12] "HALLMARK_DNA_REPAIR"                       
[13] "HALLMARK_E2F_TARGETS"                      
[14] "HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION"
[15] "HALLMARK_ESTROGEN_RESPONSE_EARLY"          
[16] "HALLMARK_ESTROGEN_RESPONSE_LATE"           
[17] "HALLMARK_FATTY_ACID_METABOLISM"            
[18] "HALLMARK_G2M_CHECKPOINT"                   
[19] "HALLMARK_GLYCOLYSIS"                       
[20] "HALLMARK_HEDGEHOG_SIGNALING"               
[21] "HALLMARK_HEME_METABOLISM"                  
[22] "HALLMARK_HYPOXIA"                          
[23] "HALLMARK_IL2_STAT5_SIGNALING"              
[24] "HALLMARK_IL6_JAK_STAT3_SIGNALING"          
[25] "HALLMARK_INFLAMMATORY_RESPONSE"            
[26] "HALLMARK_INTERFERON_ALPHA_RESPONSE"        
[27] "HALLMARK_INTERFERON_GAMMA_RESPONSE"        
[28] "HALLMARK_KRAS_SIGNALING_DN"                
[29] "HALLMARK_KRAS_SIGNALING_UP"                
[30] "HALLMARK_MITOTIC_SPINDLE"                  
[31] "HALLMARK_MTORC1_SIGNALING"                 
[32] "HALLMARK_MYC_TARGETS_V1"                   
[33] "HALLMARK_MYC_TARGETS_V2"                   
[34] "HALLMARK_MYOGENESIS"                       
[35] "HALLMARK_NOTCH_SIGNALING"                  
[36] "HALLMARK_OXIDATIVE_PHOSPHORYLATION"        
[37] "HALLMARK_P53_PATHWAY"                      
[38] "HALLMARK_PANCREAS_BETA_CELLS"              
[39] "HALLMARK_PEROXISOME"                       
[40] "HALLMARK_PI3K_AKT_MTOR_SIGNALING"          
[41] "HALLMARK_PROTEIN_SECRETION"                
[42] "HALLMARK_REACTIVE_OXYGEN_SPECIES_PATHWAY"  
[43] "HALLMARK_SPERMATOGENESIS"                  
[44] "HALLMARK_TGF_BETA_SIGNALING"               
[45] "HALLMARK_TNFA_SIGNALING_VIA_NFKB"          
[46] "HALLMARK_UNFOLDED_PROTEIN_RESPONSE"        
[47] "HALLMARK_UV_RESPONSE_DN"                   
[48] "HALLMARK_UV_RESPONSE_UP"                   
[49] "HALLMARK_WNT_BETA_CATENIN_SIGNALING"       
[50] "HALLMARK_XENOBIOTIC_METABOLISM"

In [9]:
#seperate table based on gs_name (dataframe becomes a list of smaller dataframes based on gs_name)
h_sets_list <- split(h_gene_sets, h_gene_sets$gs_name)
head(h_sets_list$HALLMARK_ADIPOGENESIS)

gs_cat,gs_subcat,gs_name,gene_symbol,entrez_gene,ensembl_gene,human_gene_symbol,human_entrez_gene,human_ensembl_gene,gs_id,gs_pmid,gs_geoid,gs_exact_source,gs_url,gs_description
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
H,,HALLMARK_ADIPOGENESIS,ABCA1,19,ENSG00000165029,ABCA1,19,ENSG00000165029,M5905,26771021,,,,Genes up-regulated during adipocyte differentiation (adipogenesis).
H,,HALLMARK_ADIPOGENESIS,ABCB8,11194,ENSG00000197150,ABCB8,11194,ENSG00000197150,M5905,26771021,,,,Genes up-regulated during adipocyte differentiation (adipogenesis).
H,,HALLMARK_ADIPOGENESIS,ACAA2,10449,ENSG00000167315,ACAA2,10449,ENSG00000167315,M5905,26771021,,,,Genes up-regulated during adipocyte differentiation (adipogenesis).
H,,HALLMARK_ADIPOGENESIS,ACADL,33,ENSG00000115361,ACADL,33,ENSG00000115361,M5905,26771021,,,,Genes up-regulated during adipocyte differentiation (adipogenesis).
H,,HALLMARK_ADIPOGENESIS,ACADM,34,ENSG00000117054,ACADM,34,ENSG00000117054,M5905,26771021,,,,Genes up-regulated during adipocyte differentiation (adipogenesis).
H,,HALLMARK_ADIPOGENESIS,ACADS,35,ENSG00000122971,ACADS,35,ENSG00000122971,M5905,26771021,,,,Genes up-regulated during adipocyte differentiation (adipogenesis).


In [14]:
#save as a excel file (each sheet will be a seperate gene set/ dataframe)
library(openxlsx)

# Assign numeric sheet names (keeping it as the gene_set names made the sheet names too long)
numeric_names <- paste0("Sheet", seq_along(h_sets_list))

# Write to Excel
write.xlsx(h_sets_list, file = "hallmark_sets.xlsx", sheetName = numeric_names)